# Movie Recommendation Function

This project uses the movielens dataset which can be found in the folder this notebook is contained in.

In [1]:
import pandas as pd 
import numpy as np 

Clean Movie Data

In [2]:
moviedata = pd.read_csv('file:///Volumes/WORK/Data%20Science/practice%20data%20sets/Movies/movies.csv')
moviedata['Year'] = moviedata.title.str.extract('(\(\d\d\d\d\))', expand = False)
moviedata['Year'] = moviedata.Year.str.extract('(\d\d\d\d)', expand = False)

moviedata['title'] = moviedata.title.str.replace('(\(\d\d\d\d\))','')
moviedata['title'] = [x.strip() for x in moviedata.title]
moviedata = moviedata.drop('Year', axis = 1)

moviedata.head(100)


<ipython-input-2-30bf0b4413ec>:5: FutureWarning: The default value of regex will change from True to False in a future version.
  moviedata['title'] = moviedata.title.str.replace('(\(\d\d\d\d\))','')


,movieId,title,genres
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji,Adventure|Children|Fantasy
2,3,Grumpier Old Men,Comedy|Romance
3,4,Waiting to Exhale,Comedy|Drama|Romance
4,5,Father of the Bride Part II,Comedy
...,...,...,...
95,97,"Hate (Haine, La)",Crime|Drama
96,98,Shopping,Action|Thriller
97,99,Heidi Fleiss: Hollywood Madam,Documentary
98,100,City Hall,Drama|Thriller


find ratings data

In [7]:
ratingsdata = pd.read_csv('file:///Volumes/WORK/Data%20Science/practice%20data%20sets/movies/ratings.csv')

ratingsdata = ratingsdata.drop('timestamp', axis = 1)

ratingsdata.head()

,userId,movieId,rating
0,1,169,2.5
1,1,2471,3.0
2,1,48516,5.0
3,2,2571,3.5
4,2,109487,4.0


## Simple Recommender

In [238]:
import time
def recommender_simple(movies , ratings):
    '''A very simple recommendation system that works in a collaberative filtering manner
    
    instead of finding the correlation between users like you normally would, 
    the similarity here is just the user ratings - your rating.
    The closer to zero between your rating group and the user rating group, the greater the similarity'''
    
    input_movies = pd.DataFrame({'title': movies , 'rating': ratings})
    input_id = moviedata[moviedata['title'].isin(input_movies['title'].to_list())]

    input_movies = pd.merge(input_id, input_movies)
    input_movies = input_movies.drop('genres', axis = 1)

    user_ratings_for_movies = ratingsdata[ratingsdata['movieId'].isin(input_movies['movieId'].to_list())]

    grouped_users = user_ratings_for_movies.groupby(['userId'])
    #sort users by how many of the films they have rated matches the number of films you 
    grouped_users = sorted(grouped_users, key = lambda x: len(x[1]), reverse = True)
    grouped_users = grouped_users[0:100]

    similarity_for_users = {}

    for name, group in grouped_users:
        group = group.sort_values(by = 'movieId').reset_index()
        input_movies = input_movies.sort_values(by = 'movieId')
        
        
        summ = 0
        for user,you in zip(input_movies.merge(group, on = 'movieId', how = 'right')['rating_y'],input_movies.merge(group, on = 'movieId', how = 'right')['rating_x']):
            summ += ((user - you)/2)**2
            
        
        
        similarity_for_users[name] = summ


    



    similarity_for_users = pd.DataFrame({'user_id': similarity_for_users.keys() , 'similarity': similarity_for_users.values()})\

    similarity_for_users['absolute'] = similarity_for_users.similarity.apply(lambda x: abs(x))

    similarity_for_users = similarity_for_users.sort_values(by = 'absolute')
    
    
    films_to_recommend = ratingsdata[(ratingsdata.userId == similarity_for_users.iloc[0,0]) & (ratingsdata['movieId'].isin(input_movies['movieId'].to_list()) == False)]\
    .sort_values(by = 'rating', ascending = False)[:10]
    
    for number, x in enumerate(moviedata[moviedata['movieId'].isin(films_to_recommend['movieId'].to_list())].title):
        print(f'Recommendation {number+1}: '+ x.upper())
        time.sleep(2)

    
    
    
    

Search for films you like that may be contained in the dataset so that you may use them in the recomendation

In [249]:
def search_films(film_search):
    list1 = list(moviedata.title.unique())
    return [x for x in list1 if film_search in x]

In [250]:
search_films(film_search = input())

Star Wars


['Star Wars: Episode IV - A New Hope',
 'Star Wars: Episode V - The Empire Strikes Back',
 'Star Wars: Episode VI - Return of the Jedi',
 'Star Wars: Episode I - The Phantom Menace',
 'Star Wars: Episode II - Attack of the Clones',
 'Star Wars: Episode III - Revenge of the Sith',
 'Star Wars: The Clone Wars',
 "Empire of Dreams: The Story of the 'Star Wars' Trilogy",
 "Star Wars Uncut: Director's Cut",
 'Star Wars: Threads of Destiny',
 'Star Wars: Episode VII - The Force Awakens',
 'The Star Wars Holiday Special',
 'Robot Chicken: Star Wars',
 'Plastic Galaxy: The Story of Star Wars Toys']

list the films you like/dont like to watch

In [251]:
films_i_have_watched = ['Star Wars: Episode IV - A New Hope',
 'Star Wars: Episode V - The Empire Strikes Back',
 'Star Wars: Episode VI - Return of the Jedi',
 'Star Wars: Episode I - The Phantom Menace',
 'Star Wars: Episode II - Attack of the Clones',
 'Star Wars: Episode III - Revenge of the Sith',
 'Star Wars: The Clone Wars',
 "Empire of Dreams: The Story of the 'Star Wars' Trilogy",
 "Star Wars Uncut: Director's Cut",
 'Star Wars: Threads of Destiny',
 'Star Wars: Episode VII - The Force Awakens']



rate them in the order you gave above

In [252]:
what_i_rate_them = np.random.randint(5,6,len(films_i_have_watched)) #here i have given random ratings for testing
what_i_rate_them

array([5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5])

In [240]:
what_i_rate_them = [5,4,5]

plug it all into the recommender

In [253]:
recommender_simple(films_i_have_watched , what_i_rate_them)

Recommendation 1: DAY AFTER TOMORROW, THE
Recommendation 2: HARRY POTTER AND THE PRISONER OF AZKABAN
Recommendation 3: SPIDER-MAN 2
Recommendation 4: HAROLD AND KUMAR GO TO WHITE CASTLE
Recommendation 5: CASINO ROYALE
Recommendation 6: AVENGERS, THE
Recommendation 7: IN TIME
Recommendation 8: TWILIGHT SAGA: BREAKING DAWN - PART 1, THE
Recommendation 9: DARK KNIGHT RISES, THE
Recommendation 10: SHERLOCK HOLMES: A GAME OF SHADOWS


# Machine Learning Recommender Using Scikit Surprise

## Making the recommender ML

NB - to go straight to the function in order to see your recommendations, head to the bottom of the notebook

Find films to rate from the database like before

In [249]:
def search_films(film_search):
    list1 = list(moviedata.title.unique())
    return [x for x in list1 if film_search in x]

In [250]:
search_films(film_search = input())

Star Wars


['Star Wars: Episode IV - A New Hope',
 'Star Wars: Episode V - The Empire Strikes Back',
 'Star Wars: Episode VI - Return of the Jedi',
 'Star Wars: Episode I - The Phantom Menace',
 'Star Wars: Episode II - Attack of the Clones',
 'Star Wars: Episode III - Revenge of the Sith',
 'Star Wars: The Clone Wars',
 "Empire of Dreams: The Story of the 'Star Wars' Trilogy",
 "Star Wars Uncut: Director's Cut",
 'Star Wars: Threads of Destiny',
 'Star Wars: Episode VII - The Force Awakens',
 'The Star Wars Holiday Special',
 'Robot Chicken: Star Wars',
 'Plastic Galaxy: The Story of Star Wars Toys']

In [251]:
films_i_have_watched = ['Star Wars: Episode IV - A New Hope',
 'Star Wars: Episode V - The Empire Strikes Back',
 'Star Wars: Episode VI - Return of the Jedi',
 'Star Wars: Episode I - The Phantom Menace',
 'Star Wars: Episode II - Attack of the Clones',
 'Star Wars: Episode III - Revenge of the Sith',
 'Star Wars: The Clone Wars',
 "Empire of Dreams: The Story of the 'Star Wars' Trilogy",
 "Star Wars Uncut: Director's Cut",
 'Star Wars: Threads of Destiny',
 'Star Wars: Episode VII - The Force Awakens']



rate them in the order you gave above

In [252]:
what_i_rate_them = np.random.randint(5,6,len(films_i_have_watched)) #here i have given random ratings for testing
what_i_rate_them

array([5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5])

In [240]:
what_i_rate_them = [5,4,5]

In [273]:
dataset = ratingsdata.copy()
dataset.head()

,userId,movieId,rating
0,1,169,2.5
1,1,2471,3.0
2,1,48516,5.0
3,2,2571,3.5
4,2,109487,4.0


In [274]:
data_w_your_ratings = pd.concat([dataset,pd.DataFrame(np.array([[*['you' for x in range(len(what_i_rate_them))]],
                       [*films_i_have_watched], [*what_i_rate_them]]).T, columns = ['userId','movieId','rating'])] , axis = 0).reset_index()

In [276]:
data_w_your_ratings.drop('index', inplace = True, axis = 1)

In [256]:
#install scikit surprise

# !pip install scikit-surprise


In [257]:
import surprise as sur

In [258]:
reader = sur.Reader(rating_scale = (0,5))

In [280]:
data1 = sur.Dataset.load_from_df(data_w_your_ratings[['userId', 'movieId', 'rating']], reader)

### cross validation, train test split and grid search

train test split

In [281]:
import random 

raw_ratings = data1.raw_ratings
random.seed(3)

random.shuffle(raw_ratings)


threshold = (int(.9 * len(raw_ratings)))

A_raw_ratings = raw_ratings[:threshold]
B_raw_ratings = raw_ratings[threshold:] #making test set

data1.raw_ratings = A_raw_ratings #this is now the train set

instatiate the grid search

In [ ]:
print('grid search...')


param_grid = {'n_epochs': [5,10], 'lr_all':[0.002, 0.005]}

grid_search = sur.model_selection.GridSearchCV(sur.SVD,
                                              param_grid,
                                              measures = ['rmse'],
                                              cv = 3,
                                              refit = True)

grid_search.fit(data1)

algo = grid_search.best_estimator['rmse']



trainset = data1.build_full_trainset()
algo.fit(trainset)

#training acuracy
trainset_build = trainset.build_testset()
predictions_train = algo.test(trainset_build)

print('Training Score: ', sur.accuracy.rmse(predictions_train))



#testing accuracy
testset = data1.construct_testset(B_raw_ratings)
predictions_tets = algo.test(testset)

print('Test Score (rated items): ', sur.accuracy.rmse(predictions_test))




#testing on movies that users havnt seen
no_ratings = trainset.build_anti_testset()
predictions_no_ratings = algo.test(no_ratings)

print('Test score (unseen items): ', sur.accuracy.rmse(predictions_no_ratings, verbose = False))










grid search...


Making the recomendation with the predicted ratings for eah user

In [ ]:
users = []
items = []
average = []
prediction = []

for x in predictions_no_ratings:
    
    users.append(x.uid)
    items.append(x.iid)
    average.append(x.r_ui)
    prediction.append(x.est)
    
pred_df = pd.DataFrame({'users': users ,
                       'items': items,
                       'average': average,
                       'prediction' : prediction})

In [ ]:
recomendations = defaultdict()
for user , items in pred_df.groupby('users')[['items' , 'prediction']]:
    items = items.sort_values(by = 'prediction', ascending = False).head(10)
    recomendations[user] = [(x[0] , x[1]) for x in zip(items['items'] , items['prediction'])]
    

In [ ]:
for number, x in enumerate(recomendations['you']):
    print(f'Recommendion {number}: ', moviedata.title[list(moviedata.movieId).index(x)].upper())

### Function for ML Recommender

**Search**

In [249]:
def search_films(film_search):
    list1 = list(moviedata.title.unique())
    return [x for x in list1 if film_search in x]

In [250]:
search_films(film_search = input())

Star Wars


['Star Wars: Episode IV - A New Hope',
 'Star Wars: Episode V - The Empire Strikes Back',
 'Star Wars: Episode VI - Return of the Jedi',
 'Star Wars: Episode I - The Phantom Menace',
 'Star Wars: Episode II - Attack of the Clones',
 'Star Wars: Episode III - Revenge of the Sith',
 'Star Wars: The Clone Wars',
 "Empire of Dreams: The Story of the 'Star Wars' Trilogy",
 "Star Wars Uncut: Director's Cut",
 'Star Wars: Threads of Destiny',
 'Star Wars: Episode VII - The Force Awakens',
 'The Star Wars Holiday Special',
 'Robot Chicken: Star Wars',
 'Plastic Galaxy: The Story of Star Wars Toys']

In [251]:
films_i_have_watched = ['Star Wars: Episode IV - A New Hope',
 'Star Wars: Episode V - The Empire Strikes Back',
 'Star Wars: Episode VI - Return of the Jedi',
 'Star Wars: Episode I - The Phantom Menace',
 'Star Wars: Episode II - Attack of the Clones',
 'Star Wars: Episode III - Revenge of the Sith',
 'Star Wars: The Clone Wars',
 "Empire of Dreams: The Story of the 'Star Wars' Trilogy",
 "Star Wars Uncut: Director's Cut",
 'Star Wars: Threads of Destiny',
 'Star Wars: Episode VII - The Force Awakens']



**Rate**

In [252]:
what_i_rate_them = np.random.randint(5,6,len(films_i_have_watched)) #here i have given random ratings for testing
what_i_rate_them

array([5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5])

In [240]:
what_i_rate_them = [5,4,5]

**Function that uses a singular value decomposition algorith trained on the data above**

In [ ]:
def ml_recommend(movies , ratings):
    dataset = ratingsdata.copy()
    
    
    data_w_your_ratings = pd.concat([dataset,pd.DataFrame(np.array([[*['you' for x in range(len(what_i_rate_them))]],
                       [*films_i_have_watched], [*what_i_rate_them]]).T, columns = ['userId','movieId','rating'])] , axis = 0).reset_index()
    data_w_your_ratings.drop('index', inplace = True, axis = 1)
    
    
    data1 = sur.Dataset.load_from_df(data_w_your_ratings[['userId', 'movieId', 'rating']], sur.Reader(rating_scale = (0,5)))
    
    
    random.seed(3)

    random.shuffle(data1.raw_ratings)
    
    #train test split
    threshold = (int(.9 * len(raw_ratings)))

    A_raw_ratings = raw_ratings[:threshold]
    B_raw_ratings = raw_ratings[threshold:] #making test set

    data1.raw_ratings = A_raw_ratings #this is now the train set
    
    #algorithm
    algo = grid_search.best_estimator['rmse']
    trainset = data1.build_full_trainset()
    algo.fit(trainset)
    
    #testing on movies that users havnt seen
    no_ratings = trainset.build_anti_testset()
    predictions_no_ratings = algo.test(no_ratings)

    
    users = []
    items = []
    average = []
    prediction = []

    for x in predictions_no_ratings:

        users.append(x.uid)
        items.append(x.iid)
        average.append(x.r_ui)
        prediction.append(x.est)

    pred_df = pd.DataFrame({'users': users ,
                           'items': items,
                           'average': average,
                           'prediction' : prediction})

    recomendations = defaultdict()
    for user , items in pred_df.groupby('users')[['items' , 'prediction']]:
        items = items.sort_values(by = 'prediction', ascending = False).head(10)
        recomendations[user] = [(x[0] , x[1]) for x in zip(items['items'] , items['prediction'])]


    for number, x in enumerate(recomendations['you']):
        print(f'Recommendion {number}: ', moviedata.title[list(moviedata.movieId).index(x)].upper())

    
    

In [4]:
# def collaberative_recommender(movies , ratings):
#     input_movies = pd.DataFrame({'title': movies , 'rating': ratings})
#     input_id = moviedata[moviedata['title'].isin(input_movies['title'].to_list())]
    
#     input_movies = pd.merge(input_id, input_movies)
#     input_movies = input_movies.drop('genres', axis = 1)
    
#     user_ratings_for_movies = ratingsdata[ratingsdata['movieId'].isin(input_movies['movieId'].to_list())]
    
#     grouped_users = user_ratings_for_movies.groupby(['userId'])
#     grouped_users = sorted(grouped_users, key = lambda x: len(x[1]), reverse = True)
#     grouped_users = grouped_users[0:100]
    
#     pearsoncorrelation_for_users = {}
    
#     for name, group in grouped_users:
#         group = group.sort_values(by = 'movieId')
#         input_movies = input_movies.sort_values(by = 'movieId')
#         nratings = len(group)
        
    
#         our_usergroup_df_for_y_correl = input_movies[input_movies['movieId'].isin(group['movieId'])]
#         our_usergroupratings_for_y_correl = our_usergroup_df_for_y_correl['rating'].to_list()
        
#         previoususerratings_for_x_correl = group['rating'].to_list()
        
#         Sxx = sum([i**2 for i in previoususerratings_for_x_correl]) - pow(sum(previoususerratings_for_x_correl),2)/float(nratings)
#         Syy = sum([i**2 for i in our_usergroupratings_for_y_correl]) - pow(sum(our_usergroupratings_for_y_correl),2)/float(nratings)
#         Sxy = sum([a*b for a,b in zip(previoususerratings_for_x_correl,our_usergroupratings_for_y_correl)]) - sum(previoususerratings_for_x_correl) * sum(our_usergroupratings_for_y_correl)/nratings
        
#         if Sxx == 0 or Syy == 0:
#             pearsoncorrelation_for_users[name] = 0
#         else:
#             pearsoncorrelation_for_users[name] = Sxy/sqrt(Sxx*Syy)

#     pearsondf = pd.DataFrame.from_dict(pearsoncorrelation_for_users , orient = 'index')
#     pearsondf.columns = ['Similarity Index']
#     pearsondf['userId'] = pearsondf.index
#     pearsondf.index = range(len(pearsondf))
    
#     topusers = pearsondf.sort_values(by = 'Similarity Index', ascending = False)
#     topusers_30 = topusers[0:30]
#     topusers_30_ratings = topusers_30.merge(ratingsdata, left_on = 'userId', right_on = 'userId', how = 'inner')
#     topusers_30_ratings['weighted rating'] = topusers_30_ratings['rating']*topusers_30_ratings['Similarity Index']
#     topusers_30_ratings.head()
    
#     weighted_rating_sum_of_each_movie = topusers_30_ratings.groupby('movieId').sum()[['Similarity Index', 'weighted rating']]
#     weighted_rating_sum_of_each_movie.columns = ['Similarity Index Total', 'sum weighted rating']
#     weighted_rating_sum_of_each_movie['Movie Score'] = weighted_rating_sum_of_each_movie['sum weighted rating']/weighted_rating_sum_of_each_movie['Similarity Index Total']
#     weighted_rating_sum_of_each_movie['movieId'] =  weighted_rating_sum_of_each_movie.index
#     weighted_rating_sum_of_each_movie =  weighted_rating_sum_of_each_movie.sort_values(by = 'Movie Score', ascending = False)

#     movies_to_recommend = weighted_rating_sum_of_each_movie[:5]
#     movies_to_recommend = moviedata[moviedata['movieId'].isin(movies_to_recommend['movieId'])]
#     movies_to_recommend.index = range(1,len(movies_to_recommend.title)+1)
#     movies_to_recommend = movies_to_recommend.title.to_frame()
    
#     return movies_to_recommend


        